In [3]:
# Import the libraries needed
from bs4 import BeautifulSoup, SoupStrainer
from collections import Counter
# import httplib2
import itertools
import matplotlib as plt
import nltk
from nltk.corpus import stopwords
from nltk.probability import FreqDist
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
import pandas as pd 
import random
import spacy
from spacy import displacy
from spacy.lang.en import English
import urllib.request
from urllib.request import urlopen, Request
import random
import re
import requests

In [4]:
nlp = spacy.load("en_core_web_sm")

## Importing Dataset

In [5]:
reviews = pd.read_json('../data/reviewSelected100.json', encoding='ISO-8859-1', lines=True)

### Writing Style

#### Getting SOF article data (originally random, source code in get_urls.py)

In [6]:
page1 = requests.get('https://stackoverflow.com/questions/5652693')
soup1 = BeautifulSoup(page1.content, "html.parser")
text = list(soup1.find_all("p"))
sof_text1 = [txt.get_text() for txt in text]

In [7]:
page1 = requests.get('https://stackoverflow.com/questions/52832519')
soup1 = BeautifulSoup(page1.content, "html.parser")
text = list(soup1.find_all("p"))
sof_text2 = [txt.get_text() for txt in text]

#### Getting HWZ article data (originally random, source code in get_urls.py)

In [8]:
page1 = requests.get('https://www.hardwarezone.com.sg/feature-apple-watchos-8-takes-first-serious-step-outdoor-cycling-fitness-workout')
soup1 = BeautifulSoup(page1.content, "html.parser")
text = list(soup1.find_all("p"))
hwz_text1 = [txt.get_text() for txt in text]

In [9]:
page1 = requests.get('https://www.hardwarezone.com.sg/feature-apple-iphone-13-iphone-13-pro-2021-review-singapore-price-specs')
soup1 = BeautifulSoup(page1.content, "html.parser")
text = list(soup1.find_all("p"))
hwz_text2 = [txt.get_text() for txt in text]

#### Getting CNA article data (originally random, source code in get_urls.py)

In [10]:
cna_text1 = ['\n\n      Sport\n  \n',
 '\n\n      Sport\n  \n',
 "Olympique de Marseille's Cengiz Under in action with Lazio's Adam Marusic and Danilo Cataldi. (Photo: REUTERS/Alberto Lingria)",
 "ROME:\xa0Lazio and Olympique Marseille drew 0-0 to keep alive their hopes of reaching the Europa League knockout stages and Red Star Belgrade stayed on course with a 1-1 draw at Denmark's Midtjylland on Thursday.",
 'Chances went begging at both ends as Lazio striker Ciro Immobile hit the woodwork and had a goal scrapped for marginal offside after a VAR check while Marseille goalkeeper Thomas Strakosha produced a string of saves in Rome.',
 'Lazio are second in Group E on four points from three games, behind Galatasaray on goal difference with the Turkish side visiting bottom team Lokomotiv Moscow later on Thursday. Third-placed Marseille have three.',
 'Red Star, the 1991 European Cup winners, took the lead against the run of play against Midtjylland in Group F thanks to a 58th-minute volley from midfielder Mirko Ivanic.',
 "The home side missed chances in a lop-sided first half as goalkeeper Milan Borjan kept the visitors afloat but Nikolas Dyhr equalised in the 78th minute after Victor Lind's shot cannoned off the woodwork.",
 "Red Star stayed top on seven points from three games, one ahead of Portugal's Braga who won 1-0 at Bulgaria's Ludogorets. Midtjylland have two points and Ludogorets one.",
 "Rapid Vienna rekindled their hopes of advancing with a 2-1 home win over Dinamo Zagreb, putting them level with the Croatian champions on three points behind Group H leaders West Ham United who host Belgium's Genk later.",
 'Marco Gruell fired Rapid ahead and defender Maximilian Hoffman sealed the win after Mislav Orsic had equalised in an action-packed first half.',
 'Royal Antwerp took an early lead before fighting back to earn a 2-2 draw at Fenerbahce, whose Ecuador winger Enner Valencia scored twice and missed a penalty in Group D.',
 'The result did neither side any favours as they stayed behind the top two, with leaders Olympiakos visiting second-placed Eintracht Frankfurt later.',
 "In Group G, front-runners Bayer Leverkusen and Real Betis stayed on course to advance after their 1-1 draw kept them level on seven points ahead of third-placed Celtic, who have three after Tuesday's 2-0 home win over Ferencvaros.",
 "\n      This service is not intended for persons residing in the E.U. By clicking subscribe, I agree to receive news updates and promotional material from Mediacorp and Mediacorp's partners. \n  ",
 ' Copyright© Mediacorp 2021. Mediacorp Pte Ltd. All rights reserved. ',
 "We know it's a hassle to switch browsers but we want your experience with CNA to be fast, secure and the best it can possibly be.",
 'To continue, upgrade to a supported browser or, for the finest experience, download the mobile app.',
 'Upgraded but still having issues? Contact us']

In [11]:
cna_text2 = ['\n\n      World\n  \n',
 '\n\n      World\n  \n',
 'A broken-off minaret of Xinqu Mosque lies near a Chinese national flag near the house of worship in Changji outside Urumqi, Xinjiang Uyghur Autonomous Region, China, May 6, 2021. (Photo: REUTERS/Thomas Peter)',
 "UNITED NATIONS: Dozens of countries kept up pressure on China at the United Nations on Thursday (Oct 21) over accusations of rights abuses against Uyghur Muslims and other minorities in the country's Xinjiang region, sparking an angry response from China's UN\xa0envoy.",
 'Confrontations over Xinjiang between China and mostly Western countries, including the United States, have become a regular occurrence at both the UN\xa0General Assembly in New York and the UN\xa0Human Rights Council in Geneva.',
 'In a statement - part of which was read by French UN\xa0Ambassador Nicolas de Rivière to the General Assembly rights committee - 43 countries voiced concern about increasing "reports of widespread and systematic human rights violations," accusing China of detaining more than 1 million people in camps.',
 'They said the abuse accusations included "reports documenting torture or cruel, inhuman and degrading treatment or punishment, forced sterilisation, sexual and gender-based violence, and forced separation of children".',
 'Beijing denies all allegations of abuse of Uyghurs and describes the camps as vocational training facilities to combat religious extremism.',
 "China's UN\xa0ambassador, Zhang Jun, hit back in a statement partly read to the rights committee on Thursday.",
 '"To the US\xa0and a few other countries: Your desperate attempts to cover up your own terrible human rights record will not work," he said.',
 '"No matter how many times repeated, lies are still lies," he added. "You are using human rights as a pretext for political\xa0manoeuvring to provoke confrontation."',
 'UN\xa0human rights chief Michelle Bachelet has been negotiating access to Xinjiang since September 2018. She told the Human Rights Council in June that she hoped to agree on terms for a visit this year to examine reports of serious violations against Muslim Uyghurs.',
 'At the Human Rights Council in June, more than 40 countries urged China to give Bachelet immediate access.',
 "\n      This service is not intended for persons residing in the E.U. By clicking subscribe, I agree to receive news updates and promotional material from Mediacorp and Mediacorp's partners. \n  ",
 ' Copyright© Mediacorp 2021. Mediacorp Pte Ltd. All rights reserved. ',
 "We know it's a hassle to switch browsers but we want your experience with CNA to be fast, secure and the best it can possibly be.",
 'To continue, upgrade to a supported browser or, for the finest experience, download the mobile app.',
 'Upgraded but still having issues? Contact us']

##### Cleaning SOF text data

In [12]:
sof_text1 = sof_text1[4:]
sof_text1 = sof_text1[:-10]
temp_list = []
for line in sof_text1:
    temp_list += sent_tokenize(line)
sof_text1 = temp_list

In [13]:
sof_text1

["Let's say I have the XQuery code below:",
 'Can I use a counter, to count how many my code will enter inside the second for loop?',
 'I tried this:',
 'but I got compile errors.',
 'I also I need to sum some constraints like this:',
 "but of course this didn't work either :(.",
 'Any suggestions will be highly appreciated.',
 'Also, can you suggest a good book/tutorial/site for doing such stuff?',
 "You don't need it very often, but if you really do need a counter variable inside an XQuery for expression (analogous to position() in an xsl:for-each) you can use an at variable",
 "I don't think you have understood the basics of declarative paradigm.",
 'Total count and sum would be:',
 'Partial count and sum would be:',
 'To Display counter in loop the best and easier way is to add at $pos in your for loop.',
 '$pos will work as a counter.',
 'Code Snippet :',
 'Output:',
 "Here is another solution, if you don't want to count or act on every item of a collection, but you want to count/

In [14]:
sof_text2 = sof_text2[4:]
sof_text2 = sof_text2[:-10]
temp_list = []
for line in sof_text2:
    temp_list += sent_tokenize(line)
sof_text2 = temp_list

In [15]:
sof_text2

['In Oracle SQL Developer 3.1.07 The two subqueries below run correctly on their own.',
 "All I'm trying to do is union them together but Oracle gives an ORA-12704: charcter set mismatch error.",
 "I have checked similar posts to check the UNION syntax and I don't understand where the error is coming from.",
 'The select statements are selecting fields from the same columns in both queries so there should be no data type problems.',
 'Help would be greatly appreciated!',
 'Ok so credit to my manager Mr K who actually solved this.',
 'The problem was with the THIRDPARTY column which is the same data type in both subqueries.',
 'In order to solve the problem we used CAST (field AS VARCHAR(3)) in both subqueries.',
 'We identified the problem field by stripping all but the first field out of the queries and then adding them back in one by one until it broke.',
 "What we still don't understand is why this error occured so any information on this would still be apprieciated.",
 'Here is the

##### Cleaning CNA text data

In [16]:
cna_text1 = cna_text1[2:]
cna_text1 = cna_text1[:-7]
temp_list = []
for line in cna_text1:
    temp_list += sent_tokenize(line)
cna_text1 = temp_list

In [17]:
cna_text1

["Olympique de Marseille's Cengiz Under in action with Lazio's Adam Marusic and Danilo Cataldi.",
 '(Photo: REUTERS/Alberto Lingria)',
 "ROME:\xa0Lazio and Olympique Marseille drew 0-0 to keep alive their hopes of reaching the Europa League knockout stages and Red Star Belgrade stayed on course with a 1-1 draw at Denmark's Midtjylland on Thursday.",
 'Chances went begging at both ends as Lazio striker Ciro Immobile hit the woodwork and had a goal scrapped for marginal offside after a VAR check while Marseille goalkeeper Thomas Strakosha produced a string of saves in Rome.',
 'Lazio are second in Group E on four points from three games, behind Galatasaray on goal difference with the Turkish side visiting bottom team Lokomotiv Moscow later on Thursday.',
 'Third-placed Marseille have three.',
 'Red Star, the 1991 European Cup winners, took the lead against the run of play against Midtjylland in Group F thanks to a 58th-minute volley from midfielder Mirko Ivanic.',
 "The home side missed 

In [18]:
cna_text2 = cna_text2[2:]
cna_text2 = cna_text2[:-8]
temp_list = []
for line in cna_text2:
    temp_list += sent_tokenize(line)
cna_text2 = temp_list

In [19]:
cna_text2

['A broken-off minaret of Xinqu Mosque lies near a Chinese national flag near the house of worship in Changji outside Urumqi, Xinjiang Uyghur Autonomous Region, China, May 6, 2021.',
 '(Photo: REUTERS/Thomas Peter)',
 "UNITED NATIONS: Dozens of countries kept up pressure on China at the United Nations on Thursday (Oct 21) over accusations of rights abuses against Uyghur Muslims and other minorities in the country's Xinjiang region, sparking an angry response from China's UN\xa0envoy.",
 'Confrontations over Xinjiang between China and mostly Western countries, including the United States, have become a regular occurrence at both the UN\xa0General Assembly in New York and the UN\xa0Human Rights Council in Geneva.',
 'In a statement - part of which was read by French UN\xa0Ambassador Nicolas de Rivière to the General Assembly rights committee - 43 countries voiced concern about increasing "reports of widespread and systematic human rights violations," accusing China of detaining more than

##### Cleaning HWZ text data

In [20]:
hwz_text1 = hwz_text1[:-5]
temp_list = []
for line in hwz_text1:
    temp_list += sent_tokenize(line)
hwz_text1 = temp_list

In [21]:
hwz_text1

['When Apple revealed at its California Streaming event that its watchOS 8 update (available now) would bring with it some major quality of life upgrades for its Outdoor Cycle workout, cyclists like me sat up.',
 'Up until now, tracking features on the Apple watches for cycling have been pretty limited - functions like auto-pause and ride detection that most of us cyclists take for granted on our Garmin or Wahoo computers are absent.',
 'But watchOS 8 will change all of these and then some.',
 'That Apple showed and focused on a mountain biker during the presentation is a watershed moment for cyclists using an Apple Watch.',
 'It’s possibly the only major fitness segment that the company hasn’t quite dominate like it did with other mainstream fitness areas like jogging and yoga.',
 'With cycling facing its biggest “bike boom” during the pandemic (be prepared to wait for months if you’re looking to get a foldie or a roadbike now), it’s timely for Apple to jazz up the latest watchOS with

In [22]:
hwz_text2 = hwz_text2[:-5]
temp_list = []
for line in hwz_text2:
    temp_list += sent_tokenize(line)
hwz_text2 = temp_list

In [23]:
hwz_text2

["Note: This review was first published on 21 September 2021 and it's republished now because the new iPhones are available in retail.",
 'Despite reservations about the name, Apple has stuck with what’s sensible and went with the number 13 for their latest iPhones.',
 'You might have already heard, Apple’s newest iPhones are the iPhone 13 and iPhone 13 Mini, and the iPhone 13 Pro and iPhone 13 Pro Max .',
 'No surprises, there.',
 'However, there are some profound changes compared to last year’s lineup and that could affect your purchasing decision.',
 'Sit down, grab a cuppa, this is a long one.',
 'Last year’s iPhones\xa0got\xa0a major redesign so we knew we weren’t going to be getting phones that look drastically different this year.',
 'The basic recipe for this year’s iPhones is nearly identical to last year’s models.',
 'In fact, unless you know what to look out for, you’d be hard-pressed to tell the difference between this year’s models and last year’s.',
 'They all have flat s

##### First word in sentence capitalized?

In [24]:
def first_word_cap(text):
    count=0
    uppercount=0
    for sent in text:
        if sent[0].isupper():
            uppercount+=1
        count+=1
    return uppercount/count

In [25]:
print("Fraction of first letter being capitalised for sof_text1: ", first_word_cap(sof_text1))
print("Fraction of first letter being capitalised for sof_text1: ", first_word_cap(sof_text2))
print("Fraction of first letter being capitalised for hwz_text1: ", first_word_cap(hwz_text1))
print("Fraction of first letter being capitalised for hwz_text2: ", first_word_cap(hwz_text2))
print("Fraction of first letter being capitalised for cna_text1: ", first_word_cap(cna_text1))
print("Fraction of first letter being capitalised for cna_text2: ", first_word_cap(cna_text2))

Fraction of first letter being capitalised for sof_text1:  0.8095238095238095
Fraction of first letter being capitalised for sof_text1:  1.0
Fraction of first letter being capitalised for hwz_text1:  0.8648648648648649
Fraction of first letter being capitalised for hwz_text2:  1.0
Fraction of first letter being capitalised for cna_text1:  0.9230769230769231
Fraction of first letter being capitalised for cna_text2:  0.7777777777777778


##### Length of articles?

In [26]:
print("No of sentences in sof_text1: ", len(sof_text1))
print("No of sentences in sof_text2: ", len(sof_text2))
print("No of sentences in hwz_text1: ", len(hwz_text1))
print("No of sentences in hwz_text2: ", len(hwz_text2))
print("No of sentences in cna_text1: ", len(cna_text1))
print("No of sentences in cna_text2: ", len(cna_text2))

No of sentences in sof_text1:  21
No of sentences in sof_text2:  11
No of sentences in hwz_text1:  37
No of sentences in hwz_text2:  52
No of sentences in cna_text1:  13
No of sentences in cna_text2:  9


##### Proper nouns capitalised?

In [27]:
def prop_nouns_cap(text):
    tagged = []
    uppercount = 0
    count = 0
    for sentence in text:
        tagged.append(nlp(sentence))
    for tag in tagged:
        for token in tag:
            if token.pos_ == 'PROPN':
                if token.text[0].isupper():
                    uppercount += 1
                count += 1
    return uppercount/count

In [28]:
print('Fraction of proper nouns capitalised in sof_text1: ', prop_nouns_cap(sof_text1))
print('Fraction of proper nouns capitalised in sof_text2: ', prop_nouns_cap(sof_text2))
print('Fraction of proper nouns capitalised in hwz_text1: ', prop_nouns_cap(hwz_text1))
print('Fraction of proper nouns capitalised in hwz_text2: ', prop_nouns_cap(hwz_text2))
print('Fraction of proper nouns capitalised in cna_text1: ', prop_nouns_cap(cna_text1))
print('Fraction of proper nouns capitalised in cna_text2: ', prop_nouns_cap(cna_text2))

Fraction of proper nouns capitalised in sof_text1:  0.8888888888888888
Fraction of proper nouns capitalised in sof_text2:  0.8571428571428571
Fraction of proper nouns capitalised in hwz_text1:  0.8472222222222222
Fraction of proper nouns capitalised in hwz_text2:  0.7126436781609196
Fraction of proper nouns capitalised in cna_text1:  1.0
Fraction of proper nouns capitalised in cna_text2:  0.9473684210526315


###### What kind of proper nouns used?

1. Stack Overflow

In [29]:
sof_tagged = []
sof_noun_dict = {}
for sentence in sof_text1:
    sof_tagged.append(nlp(sentence))
for sentence in sof_text2:
    sof_tagged.append(nlp(sentence))
for tagged in sof_tagged:
    for token in tagged:
        if token.pos_ in ("PROPN", "NOUN"):
            if token.text in sof_noun_dict.keys():
                sof_noun_dict[token.text] += 1
            else:
                sof_noun_dict[token.text] = 1

sof_noun_dict_sorted = sorted(sof_noun_dict.items(), key=lambda x: x[1], reverse=True)

for i in sof_noun_dict_sorted:
    print(i[0], i[1])

XQuery 4
counter 4
code 3
loop 3
subqueries 3
error 3
problem 3
field 3
variable 2
expression 2
paradigm 2
count 2
sum 2
way 2
pos 2
solution 2
Oracle 2
queries 2
data 2
type 2
compile 1
errors 1
constraints 1
course 1
suggestions 1
book 1
tutorial 1
site 1
stuff 1
position 1
xsl 1
basics 1
Display 1
Code 1
Snippet 1
Output 1
item 1
collection 1
condition 1
case 1
programming 1
counting 1
function 1
calls 1
usage 1
XPath 1
SQL 1
Developer 1
3.1.07 1
run 1
charcter 1
set 1
mismatch 1
posts 1
UNION 1
syntax 1
statements 1
fields 1
columns 1
problems 1
Help 1
credit 1
manager 1
Mr 1
K 1
THIRDPARTY 1
column 1
order 1
CAST 1
information 1


In [30]:
hwz_tagged = []
hwz_noun_dict = {}
for sentence in hwz_text1:
    hwz_tagged.append(nlp(sentence))
for sentence in hwz_text2:
    hwz_tagged.append(nlp(sentence))
for tagged in hwz_tagged:
    for token in tagged:
        if token.pos_ in ("PROPN", "NOUN"):
            if token.text in hwz_noun_dict.keys():
                hwz_noun_dict[token.text] += 1
            else:
                hwz_noun_dict[token.text] = 1

hwz_noun_dict_sorted = sorted(hwz_noun_dict.items(), key=lambda x: x[1], reverse=True)

for i in hwz_noun_dict_sorted:
    print(i[0], i[1])

Apple 26
year 21
iPhone 16
Watch 12
Pro 12
iPhones 11
cycling 10
phones 9
watchOS 8
bike 8
cyclists 7
workout 6
detection 6
’s 6
fall 6
glass 6
Blue 6
Julz 5
Mini 5
Max 5
sides 5
update 4
call 4
impact 4
falls 4
models 4
Lightning 4
USB 4
colours 4
Sierra 4
time 3
watch 3
sensors 3
feature 3
motion 3
difference 3
port 3
C 3
Midnight 3
Pink 3
shade 3
Gold 3
finish 3
quality 2
Outdoor 2
features 2
functions 2
auto 2
pause 2
Garmin 2
Wahoo 2
fitness 2
company 2
video 2
people 2
computer 2
algorithms 2
data 2
heart 2
rate 2
traffic 2
watches 2
users 2
emergency 2
services 2
Series 2
cadence 2
power 2
customers 2
level 2
meter 2
feedback 2
article 2
team 2
interest 2
StarHub 2
EasyGo 2
option 2
lineup 2
bodies 2
matte 2
backs 2
ports 2
point 2
thickness 2
battery 2
bit 2
cases 2
camera 2
notch 2
system 2
units 2
pink 2
process 2
model 2
California 1
Streaming 1
event 1
life 1
upgrades 1
Cycle 1
computers 1
mountain 1
biker 1
presentation 1
moment 1
segment 1
areas 1
jogging 1
yoga 1
boom 1


In [31]:
cna_tagged = []
cna_noun_dict = {}
for sentence in cna_text1:
    cna_tagged.append(nlp(sentence))
for sentence in cna_text2:
    cna_tagged.append(nlp(sentence))
for tagged in cna_tagged:
    for token in tagged:
        if token.pos_ in ("PROPN", "NOUN"):
            if token.text in cna_noun_dict.keys():
                cna_noun_dict[token.text] += 1
            else:
                cna_noun_dict[token.text] = 1

cna_noun_dict_sorted = sorted(cna_noun_dict.items(), key=lambda x: x[1], reverse=True)

for i in cna_noun_dict_sorted:
    print(i[0], i[1])

China 6
rights 5
UN 5
Marseille 4
Lazio 4
Thursday 4
Group 4
points 4
countries 4
Red 3
Star 3
Midtjylland 3
United 3
Xinjiang 3
Olympique 2
action 2
Photo 2
REUTERS 2
hopes 2
draw 2
woodwork 2
goal 2
goalkeeper 2
Thomas 2
games 2
side 2
lead 2
minute 2
home 2
half 2
Ludogorets 2
Rapid 2
win 2
Uyghur 2
accusations 2
General 2
Assembly 2
statement 2
committee 2
reports 2
camps 2
abuse 2
Cengiz 1
Adam 1
Marusic 1
Danilo 1
Cataldi 1
Alberto 1
Lingria 1
ROME 1
Europa 1
League 1
knockout 1
stages 1
Belgrade 1
course 1
Denmark 1
Chances 1
ends 1
striker 1
Ciro 1
Immobile 1
offside 1
VAR 1
Strakosha 1
string 1
saves 1
Rome 1
E 1
Galatasaray 1
difference 1
team 1
Lokomotiv 1
Moscow 1
European 1
Cup 1
winners 1
run 1
play 1
F 1
thanks 1
volley 1
midfielder 1
Mirko 1
Ivanic 1
chances 1
lop 1
Milan 1
Borjan 1
visitors 1
Nikolas 1
Dyhr 1
Victor 1
Lind 1
shot 1
Portugal 1
Braga 1
Bulgaria 1
Vienna 1
Dinamo 1
Zagreb 1
level 1
champions 1
H 1
leaders 1
West 1
Ham 1
Belgium 1
Genk 1
Marco 1
Gruell 1
M

#### % of passive sentences in the text

In [33]:
def passive_sents(text):
    passivecounts = 0
    for sent in text:
        if is_passive(sent):
            passivecounts += 1
    return passivecounts/len(text)

In [34]:
print("% of passive sentences in sof_text1: ", passive_sents(sof_text1))
print("% of passive sentences in sof_text2: ", passive_sents(sof_text2))
print("% of passive sentences in hwz_text1: ", passive_sents(hwz_text1))
print("% of passive sentences in hwz_text2: ", passive_sents(hwz_text2))
print("% of passive sentences in cna_text1: ", passive_sents(cna_text1))
print("% of passive sentences in cna_text2: ", passive_sents(cna_text2))

% of passive sentences in sof_text1:  0.0
% of passive sentences in sof_text2:  0.0
% of passive sentences in hwz_text1:  0.02702702702702703
% of passive sentences in hwz_text2:  0.09615384615384616
% of passive sentences in cna_text1:  0.0
% of passive sentences in cna_text2:  0.0


#### 1. Avg length of sentences 

In [35]:
def avg_sent_len(text):
    total_words = 0
    for sent in text:
        total_words += len(sent)
    return total_words/len(text)

In [36]:
print("Average length of sentences in sof_text1: ", avg_sent_len(sof_text1))
print("Average length of sentences in sof_text2: ", avg_sent_len(sof_text2))
print("Average length of sentences in hwz_text1: ", avg_sent_len(hwz_text1))
print("Average length of sentences in hwz_text2: ", avg_sent_len(hwz_text2))
print("Average length of sentences in cna_text1: ", avg_sent_len(cna_text1))
print("Average length of sentences in cna_text2: ", avg_sent_len(cna_text2))

Average length of sentences in sof_text1:  61.714285714285715
Average length of sentences in sof_text2:  87.54545454545455
Average length of sentences in hwz_text1:  146.9189189189189
Average length of sentences in hwz_text2:  100.25
Average length of sentences in cna_text1:  141.69230769230768
Average length of sentences in cna_text2:  176.88888888888889


#### 2. Short sentences in text

In [53]:
def num_short_sents(text):
    count = 0
    for sent in text:
        if len(sent) < 20:
            print(sent)
            count += 1
    return count

In [54]:
print("Count of short sentences in sof_text1: ", num_short_sents(sof_text1), "out of", len(sof_text1), "sentences")
print("Count of short sentences in sof_text2: ", num_short_sents(sof_text2), "out of", len(sof_text2), "sentences")
print("Count of short sentences in hwz_text1: ", num_short_sents(hwz_text1), "out of", len(hwz_text1), "sentences")
print("Count of short sentences in hwz_text2: ", num_short_sents(hwz_text2), "out of", len(hwz_text2), "sentences")
print("Count of short sentences in cna_text1: ", num_short_sents(cna_text1), "out of", len(cna_text1), "sentences")
print("Count of short sentences in cna_text2: ", num_short_sents(cna_text2), "out of", len(cna_text2), "sentences")

I tried this:
Code Snippet :
Output:
or even
Count of short sentences in sof_text1:  4 out of 21 sentences
Count of short sentences in sof_text2:  0 out of 11 sentences
Count of short sentences in hwz_text1:  0 out of 37 sentences
Alright, rant over.
Is it noticeable?
Count of short sentences in hwz_text2:  2 out of 52 sentences
Count of short sentences in cna_text1:  0 out of 13 sentences
Count of short sentences in cna_text2:  0 out of 9 sentences


# Formality

#### No of second-person pronoun 'you'

In [43]:
def num_sec_pronoun(text):
    count = 0 
    for sent in text:
        if 'you' in sent:
            count += 1 
    return count

In [44]:
print("Count of second person pronoun in sof_text1: ", num_sec_pronoun(sof_text1))
print("Count of second person pronoun in sof_text2: ", num_sec_pronoun(sof_text2))
print("Count of second person pronoun in hwz_text1: ", num_sec_pronoun(hwz_text1))
print("Count of second person pronoun in hwz_text2: ", num_sec_pronoun(hwz_text2))
print("Count of second person pronoun in cna_text1: ", num_sec_pronoun(cna_text1))
print("Count of second person pronoun in cna_text2: ", num_sec_pronoun(cna_text2))

Count of second person pronoun in sof_text1:  7
Count of second person pronoun in sof_text2:  0
Count of second person pronoun in hwz_text1:  12
Count of second person pronoun in hwz_text2:  5
Count of second person pronoun in cna_text1:  0
Count of second person pronoun in cna_text2:  1


In [46]:
#contractions
def percentage_of_contractions(text):
    total_words = 0
    for sent in text:
        total_words += len(sent)
    count = 0
    for sent in text:
        if "'" in sent:
            count += 1
    return count / total_words

In [47]:
print("Percentage of contractions in sof_text1: ", percentage_of_contractions(sof_text1))
print("Percentage of contractions in sof_text2: ", percentage_of_contractions(sof_text2))
print("Percentage of contractions in hwz_text1: ", percentage_of_contractions(hwz_text1))
print("Percentage of contractions in hwz_text2: ", percentage_of_contractions(hwz_text2))
print("Percentage of contractions in cna_text1: ", percentage_of_contractions(cna_text1))
print("Percentage of contractions in cna_text2: ", percentage_of_contractions(cna_text2))

Percentage of contractions in sof_text1:  0.0038580246913580245
Percentage of contractions in sof_text2:  0.003115264797507788
Percentage of contractions in hwz_text1:  0.0014716703458425313
Percentage of contractions in hwz_text2:  0.0007673124880107424
Percentage of contractions in cna_text1:  0.0027144408251900108
Percentage of contractions in cna_text2:  0.001256281407035176


In [50]:
#common singlish words adapted from wikipedia
common_singlish_words_a = ["abit","abuden","agak","ah","ahbeng","ahseng","ahlian","ayam","atas","aiya","aiyah","aiyo","aiyoh","alamak","angmoh","angpow","ar","arh"]
common_singlish_words_b = ["balik","bakkwa","belanja","bodoh","bo","boh","bochup","bojio","boleh","buay","buaypai","buaytahan","buey,""bope","bopian","barang","botak"]
common_singlish_words_c = ["chao","chaokeng","cheena","charbor","cher","chim","cheem","chio","chiobu","chiong","chope","chiongsua","cmi","chinchai"]
common_singlish_words_d = ["dabao","damn","dey","diam","dulan","dunnid","dope"]
common_singlish_words_e = ["eeyer","encik"]
common_singlish_words_f = []
common_singlish_words_g = ["gabra","gahmen","garang","geh geh","geh kiang","ger","gone-case","goondu","gostan","guailan"]
common_singlish_words_h = ["haolian","heng","horlan","hongbao","ho liao","honggan","hoseh","huat","humji"]
common_singlish_words_i = ["ini macam"]
common_singlish_words_j = ["jelak","jiak","jiak chao","jialat","jio"]
common_singlish_words_k = ["kampong","kaypoh","kayu","keesiao","kena","kns","kenasai","kiam","kiampak","kiasi","kiasu","kilat","kopi","kopitiam"]
common_singlish_words_l = ["la","lah","lor","lo","leh","le","liao","laosai","lepak","liddat","limpeh","limbu","lobang","luan"]
common_singlish_words_m = ["macam","mafan","makan","makcik","malu","mati","mata","meh","mah"]
common_singlish_words_n = ["ngeow","nia","nua","nia gong","nuah","nolor"]
common_singlish_words_o = ["obasan","obiang","orbi","ord","ordloh","orh","orh hor","oso","owadio"]
common_singlish_words_p = ["paikia","paiseh","ps","paktor","pangsai","pangseh","pangjio","pasarmalam","pia","piak","pok kai","pon","powderful","paitao"]
common_singlish_words_q = []
common_singlish_words_r = ["rabak","rabz"]
common_singlish_words_s = ["sabo","sakar","saman","sampat","sam seng","sargen","sei","sekali","shiok","sia","siah","seh","siam","sian","sien","siao","siasuey","sibei","sibeh","simi","sod","sorzai","suay","suka","suku","swee","swaku"]
common_singlish_words_t = ["tahan","taiko","tyco","tak boleh","tapao","tapau","taupok","teh","tekan","terbalik","tembalik","tiam","tio","towkay"]
common_singlish_words_u = ["ulu"]
common_singlish_words_v = []
common_singlish_words_w = ["wa","wah","walan","wahlan","walao","wahlao","wakao","wahkao","wayang"]
common_singlish_words_x = ["xia suay","xialan"]
common_singlish_words_y = ["yandao","yaya"]
common_singlish_words_z = ["zai","zao","zhabo","zhun","zhunbo","zi siao"]
common_singlish_words = common_singlish_words_a + common_singlish_words_b + common_singlish_words_c + common_singlish_words_d + common_singlish_words_e + common_singlish_words_f + common_singlish_words_g + common_singlish_words_h + common_singlish_words_i + common_singlish_words_j + common_singlish_words_k + common_singlish_words_l + common_singlish_words_m + common_singlish_words_n + common_singlish_words_o + common_singlish_words_p + common_singlish_words_q + common_singlish_words_r + common_singlish_words_s + common_singlish_words_t + common_singlish_words_u + common_singlish_words_v + common_singlish_words_w + common_singlish_words_x + common_singlish_words_y + common_singlish_words_z
len(common_singlish_words)

211

In [51]:
#singlish
def percentage_of_singlish(text):
    total_words = 0
    for sent in text:
        total_words += len(sent)
    count = 0
    for sent in text:
        if "'" in common_singlish_words:
            count += 1
    return count / total_words

In [52]:
print("Percentage of Singlish words in sof_text1: ", percentage_of_singlish(sof_text1))
print("Percentage of Singlish words in sof_text2: ", percentage_of_singlish(sof_text2))
print("Percentage of Singlish words in hwz_text1: ", percentage_of_singlish(hwz_text1))
print("Percentage of Singlish words in hwz_text2: ", percentage_of_singlish(hwz_text2))
print("Percentage of Singlish words in cna_text1: ", percentage_of_singlish(cna_text1))
print("Percentage of Singlish words in cna_text2: ", percentage_of_singlish(cna_text2))

Percentage of Singlish words in sof_text1:  0.0
Percentage of Singlish words in sof_text2:  0.0
Percentage of Singlish words in hwz_text1:  0.0
Percentage of Singlish words in hwz_text2:  0.0
Percentage of Singlish words in cna_text1:  0.0
Percentage of Singlish words in cna_text2:  0.0


Findings:
1. Length of articles
    - SOF consistently the shortest, HWZ and CNA are higher by a big margin (+30 sentences on avg)
2. Proper nouns
    - SOF has the lowest counts of capitalisation for proper nouns (expected)
3. Kind of proper nouns used 
    - SOF always uses language from programming/coding domain
    - HWZ always writes about either telco/pop culture (movies?)
    - CNA 
4. Length of sentences (CHECK THIS!!)
    - SOF always has the shortest sentences
    - HWZ and CNA has much longer sentences (approx 100 words)
5. Counts of extremely short sentences